In [8]:
import os
import re
import pandas as pd
import pickle
import collections
import numpy as np
import math
from ast import literal_eval
from time import gmtime, strftime
import re
import time
from bs4 import BeautifulSoup
from tqdm import trange, tqdm_notebook

# Scrapping
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

# Error Handling
import socket
import urllib3
import urllib.request
from urllib.request import urlopen
from urllib.parse import quote_plus
from urllib.request import urlretrieve
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
import warnings
warnings.filterwarnings('ignore')

from collections import defaultdict

In [3]:
import os

print("Current working directory : ", os.getcwd())

Current working directory :  G:\내 드라이브\Mycelebs\glowpick_scraping


In [4]:
df_brd_path = os.getcwd() + '/' + 'brd_info.csv'
df_brd = pd.read_csv(df_brd_path)
df_brd = df_brd.iloc[:, 1:]
brd_names = df_brd.loc[:, 'brand_name']

print("length of list of unique brand name : ", len(df_brd))
print("df_brd ⬇" )
df_brd[0:3]

length of list of unique brand name :  8052
df_brd ⬇


,brand_code,brand_name,category
0,https://www.glowpick.com/brands/8016,가가알로에,['스킨케어']
1,https://www.glowpick.com/brands/1265,가그린,['생활용품']
2,https://www.glowpick.com/brands/2077,가네보,"['페이스메이크업', '립메이크업', '컨투어링', '클렌징', '배쓰&바디', '..."


In [20]:
v4_path = os.getcwd() + '/' + 'V4_DF_ind_set.csv'
df_v4 = pd.read_csv(v4_path)

print('length of item list of v4: ', len(df_v4))
print("df_v4 ⬇" )
df_v4[0:3]

length of item list of v4:  75121
df_v4 ⬇


,브랜드_link,브랜드명,브랜드_id,제품_link,제품_id,제품명,제품 용량,제품 가격,유사 제품군 랭킹(int),유사 제품군 랭킹(text),...,제품 성분 구성 : 중간 위험도(갯수),제품 성분 구성 : 중간 위험도 (성분(한글)),제품 성분 구성 : 중간 위험도 (성분(영어)),제품 성분 구성 : 중간 위험도 (성분(설명)),제품 성분 구성 : 높은 위험도,제품 성분 구성 : 높은 위험도(갯수),제품 성분 구성 : 높은 위험도 (성분(한글)),제품 성분 구성 : 높은 위험도 (성분(영어)),제품 성분 구성 : 높은 위험도 (성분(설명)),제품 카테고리
0,,0720,NaN,https://www.glowpick.com/products/89920,89920,오일크러쉬 팩트,8g,"7,500원",115위,프레스드파우더,...,3개,"['미네랄오일', '디메치콘;다이메티콘', '피티에프이']","['MineralOil;ParaffinumLiquidum', 'Dimethicone...","['착향제,헤어컨디셔닝제,피부보호제,피부컨디셔닝제(유연제),피부컨디셔닝제(수분차단제...",높은위험도,0개,[],[],[],"['페이스메이크업', '피니시파우더', '프레스드파우더']"
1,,0720,NaN,https://www.glowpick.com/products/104423,104423,봄봄 반달형 휴대용 브러시,1p,"2,000원",151위,파우더브러시,...,0개,[],[],[],높은위험도,0개,[],[],[],"['뷰티툴', '브러시', '파우더브러시']"
2,,0720,NaN,https://www.glowpick.com/products/97152,97152,[단종] 걸크러쉬 마스카라,1p,"2,000원",[],[],...,0개,[],[],[],높은위험도,0개,[],[],[],"['아이메이크업', '마스카라', '볼륨마스카라']"


In [10]:
print('lenght of unique brand name in v4: ', len(list(set(df_v4.브랜드명))))

lenght of unique brand name in v4:  4134


# 브랜드별 카테고리 수집

In [20]:
#브랜드별 카테고리 수집

wd = webdriver.Chrome(ChromeDriverManager().install())
categories = []
idx = 0
total = len(df_brd)

for i in df_brd.loc[:, 'brand_code']:
    try:
        category = []
        wd.get(i)
        time.sleep(2)
        wd.find_element_by_xpath('//*[@id="default-layout"]/div/div[1]/span/div/div[2]/div[2]/button[2]').click()
        time.sleep(2)

        html = wd.page_source
        soup = soup = BeautifulSoup(html,"html.parser")

        for i in range(len(soup.findAll('div', {'class':'selector__item__div'}))):
            category_name = soup.select('.selector__item__div')[i].get_text().strip()
            category.append(category_name)
        category.pop(0) #'전체' 카테고리 제외
        categories.append(category)
        idx += 1
        print(idx, '/ ',total)
        

    except Exception as e:
        print(idx, '번 오류 : ', e)
        categories.append([])
        error.append(idx)
        idx += 1
        pass



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [/Users/kimjisoo/.wdm/drivers/chromedriver/mac64/93.0.4577.63/chromedriver] found in cache


1 /  8052
2 /  8052
3 /  8052
4 /  8052
5 /  8052
6 /  8052
7 /  8052
8 /  8052
9 /  8052
10 /  8052
11 /  8052


KeyboardInterrupt: 

# 브랜드별 제품 link, name 수집

In [16]:
dict_prd_link = {}
dict_prd_name = defaultdict(list)
no_such_element_exception = []
unknown_exception = []

In [17]:
def scrap_product_link(brd_name):
    
    try: 
        options = Options()
        ua = UserAgent()
        userAgent = ua.random
        print(userAgent)

#        options.add_argument('headless')
        options.add_argument('window-size=1920x1080')
        options.add_argument("disable-gpu")
        options.add_argument(f'user-agent={userAgent}')
        wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

        socket.setdefaulttimeout(30)

        url = 'https:/www.glowpick.com/searches/keywords'
        wd.get(url)
        time.sleep(5)

        wd.find_element_by_xpath("//*[@id='default-layout']/div/div[1]/span/div/div[2]/div[2]/button[2]").click()
        wd.maximize_window()
        time.sleep(3)

        brd_input = wd.find_element_by_xpath("//*[@id='default-layout']/div/div[2]/div/div/div/input")
        brd_input.send_keys(brd_name)
        brd_input.send_keys(Keys.ENTER)
        time.sleep(3)

        def parsing_html_with_class(parsing_info = '.products'):
            html = wd.page_source
            soup = soup = BeautifulSoup(html, "html.parser")
            interm_r = soup.select(parsing_info)
            return interm_r

        try:
            while True:
                prd_link = []
                prd_name = []

                product_lst = parsing_html_with_class()
                product_len = len(re.findall('<div class="products__product product" (.*?)', str(product_lst[0])))
                
                #while wd.find_elements_by_class_name('details__brand')[product_len].text[:len(brd_name)] == brd_name:

                element = wd.find_element_by_xpath('/html/body/div/div/div/div/main/div/section/div[2]/div/div[1]/span/div[{}]'.format(product_len - 1))
                wd.execute_script("arguments[0].scrollIntoView(true);", element)
                time.sleep(2)

                product_lst_2 = parsing_html_with_class()

                if product_lst == product_lst_2:
                    for i in range(product_len):
                        if re.sub(r'\([^)]*\)', '', wd.find_elements_by_class_name('details__brand')[i].text) == brd_name:
                            code = re.findall('<div class="products__product product" (.*?)id="(.*?)">', str(product_lst[0]))[i][1].split('-')[-1]
                            link = 'https://www.glowpick.com/products/' + code
                            prd_link.append(link)
                            dict_prd_name[brd_name].append(wd.find_elements_by_class_name('details__product')[i].text)
                    dict_prd_link[brd_name] = prd_link
                    break

        except NoSuchElementException:
            for i in range(product_len):
                if re.sub(r'\([^)]*\)', '', wd.find_elements_by_class_name('details__brand')[i].text) == brd_name:
                    code = re.findall('<div class="products__product product" (.*?)id="(.*?)">', str(product_lst[0]))[i][1].split('-')[-1]
                    link = 'https://www.glowpick.com/products/' + code
                    prd_link.append(link)
                    dict_prd_name[brd_name].append(wd.find_elements_by_class_name('details__product')[i].text)
            dict_prd_link[brd_name] = prd_link
        
    except NoSuchElementException:
        no_such_element_exception.append(brd_name)
        print("brand name: {}".format(brd_name), "NoSuchElementException")
        pass

    except Exception as e:
        unknown_exception.append(brd_name)
        print("brand name: {}".format(brd_name), e)
        pass
    

In [20]:
scrap_product_link('가그린')



====== WebDriver manager ======


Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.4 Safari/533.20.27


Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


In [5]:
brd_name = df_brd.loc[1, 'brand_name']

In [6]:
brd_name

'가그린'

In [12]:
for i in trange(len(df_brd)):
    brd_name = df_brd.loc[i, 'brand_name']
    scrap_product_link(brd_name)

  0%|                                                                                         | 0/8052 [00:00<?, ?it/s]

====== WebDriver manager ======


Mozilla/5.0 (Windows NT 6.2; rv:22.0) Gecko/20130405 Firefox/22.0


Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache
  0%|                                                                              | 1/8052 [00:25<57:26:19, 25.68s/it]

====== WebDriver manager ======


Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36


Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache
  0%|                                                                              | 2/8052 [01:11<83:20:33, 37.27s/it]

====== WebDriver manager ======


...

Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1866.237 Safari/537.36


Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache
  0%|▏                                                                            | 15/8052 [08:05<74:32:09, 33.39s/it]

====== WebDriver manager ======


Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2224.3 Safari/537.36


Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache
  0%|▏                                                                            | 15/8052 [08:22<74:48:52, 33.51s/it]
Exception ignored in: <function Service.__del__ at 0x00000212B3BA8310>
Traceback (most recent call last):
  File "C:\Users\keunb\.conda\envs\mycelebs_glowpick_cr\lib\site-packages\selenium\webdriver\common\service.py", line 176, in __del__
    self.stop()
  File "C:\Users\keunb\.conda\envs\mycelebs_glowpick_cr\lib\site-packages\selenium\webdriver\common\service.py", line 151, in stop
    self.send_remote_shutdown_command()
  File "C:\Users\keunb\.conda\envs\mycelebs_glowpick_cr\lib\site-packages\selenium\webdriver\common\service.py", line 132, in send_remote_shutdown_command
    if not self.is_connectable():
  File "C:\Users\keunb\.conda\envs\mycelebs_glowpick_cr\lib\site-packages\seleni

KeyboardInterrupt: 

In [21]:
dict_prd_link

{'가그린': []}

In [22]:
dict_prd_name

defaultdict(list, {})

In [39]:
dict_prd_link

{'가그린': ['https://www.glowpick.com/products/134403',
  'https://www.glowpick.com/products/134399',
  'https://www.glowpick.com/products/134398',
  'https://www.glowpick.com/products/134404',
  'https://www.glowpick.com/products/141416',
  'https://www.glowpick.com/products/134539',
  'https://www.glowpick.com/products/134520',
  'https://www.glowpick.com/products/134536',
  'https://www.glowpick.com/products/134402',
  'https://www.glowpick.com/products/134549',
  'https://www.glowpick.com/products/134401',
  'https://www.glowpick.com/products/134529',
  'https://www.glowpick.com/products/134541',
  'https://www.glowpick.com/products/134400',
  'https://www.glowpick.com/products/146721'],
 '다이브': ['https://www.glowpick.com/products/145273']}

In [40]:
for brd_name in brd_names:
    scrap_product_link(brd_name)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577


Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.2309.372 Safari/537.36


Driver [/Users/kimjisoo/.wdm/drivers/chromedriver/mac64/93.0.4577.63/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577


Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36


Driver [/Users/kimjisoo/.wdm/drivers/chromedriver/mac64/93.0.4577.63/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [/Users/kimjisoo/.wdm/drivers/chromedriver/mac64/93.0.4577.63/chromedriver] found in cache


...

Mozilla/5.0 (Windows NT 6.1; WOW64; rv:18.0)  Gecko/20100101 Firefox/18.0




====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577


Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; AS; rv:11.0) like Gecko


Driver [/Users/kimjisoo/.wdm/drivers/chromedriver/mac64/93.0.4577.63/chromedriver] found in cache


KeyboardInterrupt: 

In [69]:
brd_name = '가그린'

# options = Options()
# ua = UserAgent()
# userAgent = ua.random
# print(userAgent)

# options.add_argument('headless')
# options.add_argument('window-size=1920x1080')
# options.add_argument("disable-gpu")
# options.add_argument(f'user-agent={userAgent}')
# wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

wd = webdriver.Chrome(ChromeDriverManager().install())
#socket.setdefaulttimeout(30)

url = 'https:/www.glowpick.com/searches/keywords'
wd.get(url)
time.sleep(2)

wd.find_element_by_xpath("//*[@id='default-layout']/div/div[1]/span/div/div[2]/div[2]/button[2]").click()
wd.maximize_window()
time.sleep(2)

brd_input = wd.find_element_by_xpath("//*[@id='default-layout']/div/div[2]/div/div/div/input")
brd_input.send_keys(brd_name)
brd_input.send_keys(Keys.ENTER)
time.sleep(2)

html = wd.page_source
soup = BeautifulSoup(html, "html.parser")
product_lst = soup.select(".products")
product_len = len(re.findall('<div class="products__product product" (.*?)', str(product_lst[0])))

element = wd.find_element_by_xpath('/html/body/div/div/div/div/main/div/section/div[2]/div/div[1]/span/div[{}]'.format(product_len - 1))
#print(wd.find_elements_by_class_name('details__product').text)

find_element_by_class_name_lst = [x.text for x in wd.find_elements_by_class_name('details__brand')]

for i in range(product_len):
    print('특수문자 제거 brd_name: ', re.sub(r'\([^)]*\)', '', wd.find_elements_by_class_name('details__brand')[i].text))
    print('brd_name: ', brd_name)




====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


특수문자 제거 brd_name:  가그린 
brd_name:  가그린
특수문자 제거 brd_name:  가그린 
brd_name:  가그린
특수문자 제거 brd_name:  가그린 
brd_name:  가그린
특수문자 제거 brd_name:  가그린 
brd_name:  가그린
특수문자 제거 brd_name:  가그린 
brd_name:  가그린
특수문자 제거 brd_name:  가그린 
brd_name:  가그린
특수문자 제거 brd_name:  가그린 
brd_name:  가그린
특수문자 제거 brd_name:  가그린 
brd_name:  가그린
특수문자 제거 brd_name:  가그린 
brd_name:  가그린
특수문자 제거 brd_name:  가그린 
brd_name:  가그린
특수문자 제거 brd_name:  가그린 
brd_name:  가그린
특수문자 제거 brd_name:  가그린 
brd_name:  가그린
특수문자 제거 brd_name:  가그린 
brd_name:  가그린
특수문자 제거 brd_name:  가그린 
brd_name:  가그린
특수문자 제거 brd_name:  가그린 
brd_name:  가그린
특수문자 제거 brd_name:  닥터피엔티 
brd_name:  가그린


In [43]:
find_element_by_class_name_lst

['가그린 (garglin)',
 '가그린 (garglin)',
 '가그린 (garglin)',
 '가그린 (garglin)',
 '가그린 (garglin)',
 '가그린 (garglin)',
 '가그린 (garglin)',
 '가그린 (garglin)',
 '가그린 (garglin)',
 '가그린 (garglin)',
 '가그린 (garglin)',
 '가그린 (garglin)',
 '가그린 (garglin)',
 '가그린 (garglin)',
 '가그린 (garglin)',
 '닥터피엔티 (Dr.PNT)']

In [1]:
import ast

In [33]:
import pickle
os.getcwd()

'/Users/kimjisoo/Desktop/preprocessing'

In [35]:
with open('dict_prd_link.txt', 'wb') as fp:
    pickle.dump(dict_prd_link, fp)

In [37]:
dict_prd_name

{'결고은사람들': ['어성초 크림', '[단종]  클렌징 오일'],
 '경남제약': ['결콜라겐',
  '생유산균9C',
  '생 유산균 알파',
  '레모나 마스크 KF-AD',
  '결콜라겐 플러스',
  '결콜라겐&뮤신',
  '스트레스 케어',
  '생유산균7C',
  '칼로 스탑',
  '핸드 클린 손 소독제',
  '면역 부스터',
  '루테인 톡',
  '칼로-뷰티 콜라겐 1000'],
 '경동제약': ['애니밴드(원형)'],
 '경인 플랙스': ['황사 방역용 마스크 KF94', '황사 방역용 마스크 KF80'],
 '고귀한아름다움채': ['어성초 황금비율 두피앰플', '지청 자양 샴푸액'],
 '고려생활건강': ['티엠씨 에스 바'],
 '고려은단': ['비타민C 1000',
  '비타민C 골드 플러스',
  '비타민C 1000 이지',
  '메가도스B',
  '메가도스 C3000',
  '퓨어 알티지 오메가3',
  '비타1000 마스크팩',
  '신바이오틱스',
  '다이어트 쉐이크',
  '루테인 지아잔틴 아스타잔틴',
  '비타미녀씨',
  '퓨어 밀크씨슬',
  '퓨어 루테인',
  '프리바이오틱스',
  '듀얼 플로라 프로바이오틱스',
  '알티지 오메가3',
  '퓨어 루테인 지아잔틴'],
 '고사리숲': ['[단종]  꼬미 주니어 클렌징 폼'],
 '고세': ['클리어턴 에센스 마스크팩 - 비타민 C',
  '쓰리버터스 립 밤',
  '에스프리끄 포밍 뷰티 리퀴드 UV 파운데이션 [SPF23/PA++]',
  '프레디아 스파에메르 헤드 스파클링 에센스',
  '모이스처 스킨 리페어 에센스',
  '클리어턴 에센스 마스크팩 - 히알루론산',
  '모이스처 스킨 리페어 아이 크림',
  '프레디아 스파에메르 G 세럼',
  '클리어턴 에센스 마스크팩 - 콜라겐',
  '클리어턴 에센스 마스크팩 - 프라센타',
  '클리어턴 에센스 마스크팩 - 코엔자임 Q10',
  '클리어턴 에센스 마스크팩 - 트라네키 삼산',
  '스펀지 크리너